In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
rivers = gpd.read_file("Rivers_Data/Rivers.shp")
rivers.head()

In [ ]:
import geopandas as gpd
dams = gpd.read_file("Dams_Data/South_Africa_Dams.shp")
dams.head()